In [1]:
using Plots
using CSV
using ControlSystems

gr()

Plots.GRBackend()

In [2]:
#体を揺すって脚を出す歩容
#確定版2019/01/20

zc=0.11 #0.12
g=9.8
dt=0.0025
L=0.15 #0.16
l=0.025 #歩幅
#x=0.03
ft=0.05 #最初の全脚支持期間

#support time
Tsup=Int32(0.4/dt)
#both foot support point time
Tdl=Int32(0.0025/dt) 
#preview step
M=600 #150
            #r             c         l
real_sx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l; #第一ZMP
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l; #一度動かない瞬間を作る
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;#第二zmp
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
         
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;#stop time
    
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;#第3zmp
    
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;
        
        ] #  rcl

real_sy=[0.0; L/2; L;
    
        0.0; L/2; L;
         0.0;  L/2; L;
    
        0.0;  L/2; L; #動かない瞬間を作る
        
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;
        
        0.0;  L/2; L; #sptime
        0.0;  L/2; L;
        
        0.0;  L/2; L;#動かない期間
    
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;

        0.0;  L/2; L;
         ]

pltsx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L] #plot用
pltsy=[0.0; L/2; L]

leg_sign=[0;3;0;0;0;0;2;0;0;0;0;1;0;0;]#上げる脚とステップの関係 #left:3 right:1 center:

#初期姿勢
f_zmp_x=abs(real_sx[1]-real_sx[2])/2
f_zmp_y=L/2+0.00

e_zmp_x=abs(real_sx[1]-real_sx[2])/2+l
e_zmp_y=L/2+0.00 #追加

#Virtual support point
sx=[f_zmp_x
    real_sx[8]+abs(real_sx[4]-real_sx[8])/2+0.005+0.004
    abs(real_sx[1]-real_sx[2])/2+(l/3)
    
    abs(real_sx[1]-real_sx[2])/2+(l/3) #動かない瞬間をつくる
    
    abs(real_sx[1]-real_sx[2])/2+(l/3)+0.05 #sp time
    abs(real_sx[1]-real_sx[2])/2+(l/3)+0.05 #stop time
    real_sx[1]+(l/2)+0.012 #支持脚線の第二ZMP
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2#+0.06
    
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2#+0.05 #stop time
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2 #sp time
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2 #stop time
    
    real_sx[1]/2+l+0.00 #第三zmp 0.005
    abs(real_sx[1]-real_sx[2])/2+l+0.012 
    
    e_zmp_x]

sy=[f_zmp_y
    abs(real_sy[7]-real_sy[8])/2+0.005+0.003
    L/2
    
    L/2 #一度何もしない瞬間を作る
    
    L/2 #sp time
    L/2#stoptime
    L/2 #ZMP2
    L/2
    
    L/2#stop time
    L/2#sptime
    L/2#stoptime
    
    real_sy[2]+abs(real_sy[2]-real_sy[3])/2-0.004 #第三zmp
    L/2
    
    e_zmp_y]

vpoint_sum=length(sx)
full_leg_step_sum=vpoint_sum-1

#to generate trajectory of ZMP when both foot support point 
x_tdl_a=zeros(Float64,full_leg_step_sum)
x_tdl_b=zeros(Float64,full_leg_step_sum)
y_tdl_a=zeros(Float64,full_leg_step_sum)
y_tdl_b=zeros(Float64,full_leg_step_sum)

for i in 1:full_leg_step_sum
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:full_leg_step_sum
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#target ZMP vector
px_ref=zeros(vpoint_sum*Tsup+M+1 + full_leg_step_sum*Tdl)
py_ref=zeros(px_ref)

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            px_ref[k+step]=sx[i]
        end
    else
        for k in 1:Tsup
            px_ref[k+step]=sx[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            px_ref[s+step]=x_tdl_a[i]*(s+step)*dt+x_tdl_b[i]
        end
        step=step+Tdl
    end
end

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            py_ref[k+step]=sy[i]
        end
    else
        for k in 1:Tsup
            py_ref[k+step]=sy[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            py_ref[s+step]=y_tdl_a[i]*(s+step)*dt+y_tdl_b[i]
        end
        step=step+Tdl
    end
end

#total step
k_max=length(px_ref)-M
#computed ZMP vector
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max);

In [3]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[20000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#state vector
x=[ sx[1]
    0.0
    0.0]

y=[ sy[1]
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0];

In [4]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #to record for plot
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #to record for plot
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 

xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y

dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR
old_x=x
old_y=y
xu=0
yu=0
P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 0.0

In [5]:
for i in 1:k_max-1
    
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]    
    k+=1
    x_plot_log[k+1]=x[1] #to record for plot
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #to record for plot
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    dx=x-old_x
    dy=y-old_y
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    old_x=x
    old_y=y
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end

In [6]:
plot(t*dt,px_ref,lab="Target ZMP",linewidth=3,linecolor="black")
plot!(t*dt,x_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t*dt,px,lab="Target ZMP",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP_x[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
 
 time[ms] 
 
 
 ZMP 
 
 
 x[m] 
 
<polyline clip-path="url(#clip0002)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,219.765 39.6154,219.765 39.8608,219.765 40.1061,219.765 40.3514,219.765 40.5968,219.765 40.8421,219.765 41.0875,219.765 41.3328,219.765 41.5782,219.765 
 41.8235,219.765 42.0688,219.765 42.3142,219.765 42.5595,219.765 42.8049,219.765 43.0502,219.765 43.2956,219.765 43.5409,219.765 43.7862,219.765 44.0316,219.765 
 44.2769,219.765 44.5223,219.765 44.7676,219.765 45.0129,219.765 45.2583,219.765 45.5036,219.765 45.749,219.765 45.9943,219.765 46.2397,219.765 46.485,219.765 
 46.7303,219.765 46.9757,219.765 47.221,219.765 47.4664,219.765 47.7117,219.765 47.9571,219.765 48.2024,219.765 48.4477,219.765 48.6931,219.765 48.9384,219.765 
 49.1838,219.765 49.4291,219.765 49.6744,219.765 49.9198,219.765 50.1651,219.765 50.4105,219.765 50.6558,219.765 50.9012,219.765 51.1465,219.765 51.3918,219.765 
 51.6372,219.765 51.8825,219.765 52.1279,219.765 52.3732,219.765 52.6186,219.765 52.8639,219.765 53.1092,219.765 53.3546,219.765 53.5999,219.765 53.8453,219.765 
 54.0906,219.765 54.336,219.765 54.5813,219.765 54.8266,219.765 55.072,219.765 55.3173,219.765 55.5627,219.765 55.808,219.765 56.0533,219.765 56.2987,219.765 
 56.544,219.765 56.7894,219.765 57.0347,219.765 57.2801,219.765 57.5254,219.765 57.7707,219.765 58.0161,219.765 58.2614,219.765 58.5068,219.765 58.7521,219.765 
 58.9975,219.765 59.2428,219.765 59.4881,219.765 59.7335,219.765 59.9788,219.765 60.2242,219.765 60.4695,219.765 60.7148,219.765 60.9602,219.765 61.2055,219.765 
 61.4509,219.765 61.6962,219.765 61.9416,219.765 62.1869,219.765 62.4322,219.765 62.6776,219.765 62.9229,219.765 63.1683,219.765 63.4136,219.765 63.659,219.765 
 63.9043,219.765 64.1496,219.765 64.395,219.765 64.6403,219.765 64.8857,219.765 65.131,219.765 65.3763,219.765 65.6217,219.765 65.867,219.765 66.1124,219.765 
 66.3577,219.765 66.6031,219.765 66.8484,219.765 67.0937,219.765 67.3391,219.765 67.5844,219.765 67.8298,219.765 68.0751,219.765 68.3205,219.765 68.5658,219.765 
 68.8111,219.765 69.0565,219.765 69.3018,219.765 69.5472,219.765 69.7925,219.765 70.0378,219.765 70.2832,219.765 70.5285,219.765 70.7739,219.765 71.0192,219.765 
 71.2646,219.765 71.5099,219.765 71.7552,219.765 72.0006,219.765 72.2459,219.765 72.4913,219.765 72.7366,219.765 72.982,219.765 73.2273,219.765 73.4726,219.765 
 73.718,219.765 73.9633,219.765 74.2087,219.765 74.454,219.765 74.6994,219.765 74.9447,219.765 75.19,219.765 75.4354,219.765 75.6807,219.765 75.9261,219.765 
 76.1714,219.765 76.4167,219.765 76.6621,219.765 76.9074,219.765 77.1528,219.765 77.3981,219.765 77.6435,219.765 77.8888,219.765 78.1341,219.765 78.3795,219.765 
 78.6248,199.156 78.8702,199.156 79.1155,199.156 79.3609,199.156 79.6062,199.156 79.8515,199.156 80.0969,199.156 80.3422,199.156 80.5876,199.156 80.8329,199.156 
 81.0782,199.156 81.3236,199.156 81.5689,199.156 81.8143,199.156 82.0596,199.156 82.305,199.156 82.5503,199.156 82.7956,199.156 83.041,199.156 83.2863,199.156 
 83.5317,199.156 83.777,199.156 84.0224,199.156 84.2677,199.156 84.513,199.156 84.7584,199.156 85.0037,199.156 85.2491,199.156 85.4944,199.156 85.7397,199.156 
 85.9851,199.156 86.2304,199.156 86.4758,199.156 86.7211,199.156 86.9665,199.156 87.2118,199.156 87.4571,199.156 87.7025,199.156 87.9478,199.156 88.1932,199.156 
 88.4385,199.156 88.6839,199.156 88.9292,199.156 89.1745,199.156 89.4199,199.156 89.6652,199.156 89.9106,199.156 90.1559,199.156 90.4012,199.156 90.6466,199.156 
 90.8919,199.156 91.1373,199.156 91.3826,199.156 91.628,199.156 91.8733,199.156 92.1186,199.156 92.364,199.156 92.6093,199.156 92.8547,199.156 93.1,199.156 
 93.3454,199.156 93.5907,199.156 93.836,199.156 94.0

In [7]:
plot(t,py_ref,lab="Target ZMP y",linewidth=3,linecolor="black")
plot!(t,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t,py,lab="ZMP y",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 300 
 
 
 600 
 
 
 900 
 
 
 1200 
 
 
 1500 
 
 
 1800 
 
 
 2100 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 time[ms] 
 
 
 ZMP[m] 
 
<polyline clip-path="url(#clip0102)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,123.098 39.6154,123.098 39.8608,123.098 40.1061,123.098 40.3514,123.098 40.5968,123.098 40.8421,123.098 41.0875,123.098 41.3328,123.098 41.5782,123.098 
 41.8235,123.098 42.0688,123.098 42.3142,123.098 42.5595,123.098 42.8049,123.098 43.0502,123.098 43.2956,123.098 43.5409,123.098 43.7862,123.098 44.0316,123.098 
 44.2769,123.098 44.5223,123.098 44.7676,123.098 45.0129,123.098 45.2583,123.098 45.5036,123.098 45.749,123.098 45.9943,123.098 46.2397,123.098 46.485,123.098 
 46.7303,123.098 46.9757,123.098 47.221,123.098 47.4664,123.098 47.7117,123.098 47.9571,123.098 48.2024,123.098 48.4477,123.098 48.6931,123.098 48.9384,123.098 
 49.1838,123.098 49.4291,123.098 49.6744,123.098 49.9198,123.098 50.1651,123.098 50.4105,123.098 50.6558,123.098 50.9012,123.098 51.1465,123.098 51.3918,123.098 
 51.6372,123.098 51.8825,123.098 52.1279,123.098 52.3732,123.098 52.6186,123.098 52.8639,123.098 53.1092,123.098 53.3546,123.098 53.5999,123.098 53.8453,123.098 
 54.0906,123.098 54.336,123.098 54.5813,123.098 54.8266,123.098 55.072,123.098 55.3173,123.098 55.5627,123.098 55.808,123.098 56.0533,123.098 56.2987,123.098 
 56.544,123.098 56.7894,123.098 57.0347,123.098 57.2801,123.098 57.5254,123.098 57.7707,123.098 58.0161,123.098 58.2614,123.098 58.5068,123.098 58.7521,123.098 
 58.9975,123.098 59.2428,123.098 59.4881,123.098 59.7335,123.098 59.9788,123.098 60.2242,123.098 60.4695,123.098 60.7148,123.098 60.9602,123.098 61.2055,123.098 
 61.4509,123.098 61.6962,123.098 61.9416,123.098 62.1869,123.098 62.4322,123.098 62.6776,123.098 62.9229,123.098 63.1683,123.098 63.4136,123.098 63.659,123.098 
 63.9043,123.098 64.1496,123.098 64.395,123.098 64.6403,123.098 64.8857,123.098 65.131,123.098 65.3763,123.098 65.6217,123.098 65.867,123.098 66.1124,123.098 
 66.3577,123.098 66.6031,123.098 66.8484,123.098 67.0937,123.098 67.3391,123.098 67.5844,123.098 67.8298,123.098 68.0751,123.098 68.3205,123.098 68.5658,123.098 
 68.8111,123.098 69.0565,123.098 69.3018,123.098 69.5472,123.098 69.7925,123.098 70.0378,123.098 70.2832,123.098 70.5285,123.098 70.7739,123.098 71.0192,123.098 
 71.2646,123.098 71.5099,123.098 71.7552,123.098 72.0006,123.098 72.2459,123.098 72.4913,123.098 72.7366,123.098 72.982,123.098 73.2273,123.098 73.4726,123.098 
 73.718,123.098 73.9633,123.098 74.2087,123.098 74.454,123.098 74.6994,123.098 74.9447,123.098 75.19,123.098 75.4354,123.098 75.6807,123.098 75.9261,123.098 
 76.1714,123.098 76.4167,123.098 76.6621,123.098 76.9074,123.098 77.1528,123.098 77.3981,123.098 77.6435,123.098 77.8888,123.098 78.1341,123.098 78.3795,123.098 
 78.6248,219.625 78.8702,219.625 79.1155,219.625 79.3609,219.625 79.6062,219.625 79.8515,219.625 80.0969,219.625 80.3422,219.625 80.5876,219.625 80.8329,219.625 
 81.0782,219.625 81.3236,219.625 81.5689,219.625 81.8143,219.625 82.0596,219.625 82.305,219.625 82.5503,219.625 82.7956,219.625 83.041,219.625 83.2863,219.625 
 83.5317,219.625 83.777,219.625 84.0224,219.625 84.2677,219.625 84.513,219.625 84.7584,219.625 85.0037,219.625 85.2491,219.625 85.4944,219.625 85.7397,219.625 
 85.9851,219.625 86.2304,219.625 86.4758,219.625 86.7211,219.625 86.9665,219.625 87.2118,219.625 87.4571,219.625 87.7025,219.625 87.9478,219.625 88.1932,219.625 
 88.4385,219.625 88.6839,219.625 88.9292,219.625 89.1745,219.625 89.4199,219.625 89.6652,219.625 89.9106,219.625 90.1559,219.625 90.4012,219.625 90.6466,219.625 
 90.8919,219.625 91.1373,219.625 91.3826,219.625 91.628,219.625 91.8733,219.625 92.1186,219.625 92.364,219.625 92.6093,219.625 92.8547,219.625 93.1,219.625 
 93.3454,219.625 93.5907,219.625 93.836,219.625 94.0814,21

In [8]:
plot(x_plot_log,y_plot_log,lab="CoM",linewidth=1,linecolor="blue")
plot!(px,py,lab="ZoM",linewidth=1,linecolor="black") #追加
plot!(real_sx,real_sy,st=:scatter)
plot!(sx,sy,st=:scatter)
a=[0.15 0.15]
b=[0.19 0.19]
plot!(b,a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
 
 0.175 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
<polyline clip-path="url(#clip0202)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.19 233.28,188.19 233.28,188.191 
 233.28,188.192 233.281,188.194 233.282,188.196 233.282,188.198 233.284,188.201 233.285,188.205 233.287,188.209 233.288,188.214 233.291,188.22 233.293,188.226 
 233.296,188.234 233.299,188.242 233.303,188.252 233.307,188.262 233.311,188.274 233.316,188.287 233.321,188.301 233.327,188.316 233.333,188.333 233.34,188.351 
 233.348,188.37 233.355,188.391 233.364,188.413 233.373,188.437 233.382,188.462 233.392,188.489 233.403,188.518 233.414,188.548 233.426,188.579 233.439,188.612 
 233.452,188.647 233.466,188.684 233.48,188.722 233.496,188.762 233.511,188.804 233.528,188.847 233.545,188.893 233.563,188.94 233.581,188.989 233.6,189.039 
 233.62,189.092 233.641,189.146 233.662,189.203 233.684,189.261 233.707,189.321 233.73,189.383 233.754,189.447 233.779,189.513 233.805,189.581 233.831,189.65 
 233.858,189.722 233.886,189.796 233.915,189.873 233.945,189.951 233.975,190.031 234.006,190.114 234.038,190.198 234.071,190.286 234.105,190.375 234.14,190.467 
 234.175,190.561 234.212,190.657 234.249,190.756 234.287,190.858 234.327,190.962 234.367,191.068 234.408,191.178 234.451,191.29 234.494,191.405 234.539,191.523 
 234.585,191.644 234.631,191.768 234.679,191.896 234.729,192.026 234.779,192.16 234.831,192.297 234.884,192.438 234.939,192.583 234.995,192.731 235.052,192.883 
 235.111,193.038 235.171,193.198 235.233,193.363 235.297,193.531 235.362,193.704 235.429,193.881 235.497,194.063 235.568,194.25 235.64,194.441 235.714,194.638 
 235.791,194.84 235.869,195.047 235.949,195.26 236.031,195.479 236.116,195.703 236.202,195.933 236.291,196.169 236.383,196.411 236.476,196.66 236.573,196.915 
 236.671,197.177 236.772,197.445 236.876,197.721 236.983,198.003 237.092,198.293 237.204,198.59 237.318,198.894 237.436,199.206 237.556,199.526 237.679,199.853 
 237.806,200.189 237.935,200.532 238.067,200.884 238.203,201.244 238.341,201.612 238.483,201.988 238.628,202.373 238.776,202.766 238.927,203.167 239.082,203.577 
 239.239,203.996 239.4,204.423 239.564,204.858 239.731,205.302 239.902,205.754 240.075,206.215 240.252,206.683 240.431,207.16 240.614,207.645 240.8,208.137 
 240.988,208.638 241.18,209.145 241.374,209.66 241.571,210.182 241.77,210.711 241.972,211.247 242.177,211.788 242.383,212.336 242.592,212.889 242.803,213.447 
 243.016,214.011 243.231,214.579 243.447,215.151 243.665,215.726 243.884,216.305 244.104,216.887 244.325,217.471 244.547,218.057 244.77,218.644 244.993,219.232 
 245.217,219.821 245.44,220.409 245.663,220.997 245.887,221.584 246.11,222.169 246.332,222.753 246.554,223.335 246.775,223.915 246.995,224.491 247.214,225.064 
 247.432,225.634 247.649,226.199 247.864,226.76 248.077,227.316 248.289,227.866 248.499,228.411 248.707,228.951 248.912,229.483 249.116,230.01 249.317,230.529 
 249.516,231.041 249.712,231.546 249.905,232.043 250.096,232.532 250.284,233.013 250.47,233.486 250.652,233.951 250.832,234.407 251.008,234.854 251.182,235.292 
 251.352,235.722 251.519,236.142 251.684,236.554 251.845,236.956 252.003,237.348 252.158,237.732 252.309,238.106 252.458,238.471 252.603,238.826 252.746,239.173 
 252.885,239.509 253.021,239.837 253.154,240.155 253.283,240.464 253.41,240.764 253.534,241.055 253.655,241.336 253.772,241.609 253.887,241.873 253.999,242.127 
 254.108,242.373 254.215,242.611 254.318,242.839 254.419,243.059 254.517,243.271 254.612,243.474 254.705,243.669 254.795,243.855 254.883,244.034 254.969,244.204 
 255.051,244.367 255.132,244.521 2

In [9]:
function trj_generator(sx,xv,tf,te,dt,legnum)#xの遊脚軌道
    #
    #条件ベクトル
    xf=sx[legnum,tf]
    xe=sx[legnum,te]
    vf=xv[tf]*-1
    ve=xv[te]*-1
    A=[xf; xe; vf; ve]
    
    tf=round(tf*dt,6) #時間に戻す
    te=round(te*dt,6) #第6位で切り捨て
        
    B=[tf^3 tf^2 tf 1
        te^3 te^2 te 1
        3tf^2 2tf 1 0
        3te^2 2te 1 0]

    C=inv(B)*A

    #dt=0.01 #離散時間
    t=[i for i in tf:dt:te] #描画時間
    x=C[1]*.^(t,3)+C[2]*.^(t,2)+C[3]*t+C[4] #位置
    v=3*C[1]*.^(t,2)+2*C[2]*t+C[3] #速度
    a=6*C[1]*t+2*C[2]
    
    return x
    
end

function trj_generator_z(zh,zf,ze,ve,tf,te,dt)
    #support time 0.5
    tf=round(tf*dt,7) #時間に戻す
    te=round(te*dt,7) #第6位で切り捨て
    th=round((te-tf)/2+tf,7)
    
    zh=zf+zh
   
    A_z=[zh; ze; zf; ve]

    B_z=[th^3 th^2 th 1
        te^3 te^2 te 1
        tf^3 tf^2 tf 1
        3te^2 2te 1 0]
    C_z=inv(B_z)*A_z
    
    t=[i for i in tf:dt:te]; #描画時間
     z=C_z[1]*.^(t,3)+C_z[2]*.^(t,2)+C_z[3]*t+C_z[4] #位置
    v=3*C_z[1]*.^(t,2)+2*C_z[2]*t+C_z[3] #速度
    a=6*C_z[1]*t+2*C_z[2]
    return z
end



In [10]:
#脚先位置を重心の座標系に変換する
com_trjx=copy(x_plot_log) #重心位置
com_trjy=copy(y_plot_log)

sx_from_com=zeros(3,length(com_trjx))
sy_from_com=zeros(3,length(com_trjy))   #重心からみた支持脚位置
sz_from_com=[-zc for i in 1:length(com_trjx)*3]
sz_from_com=reshape(sz_from_com,3,length(com_trjx))

sx_from_w=reshape(real_sx,3,vpoint_sum)
sy_from_w=reshape(real_sy,3,vpoint_sum)
vz_end=-0.5 #遊脚が接地する速度 -0.1

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum) 
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
        end
    else
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
            
        end
        if(leg_sign[i]!=0)
            #脚を挙げないようにコメントアウト
            sx_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sx_from_com,xv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sy_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sy_from_com,yv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sz_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator_z(0.01,-zc+0.01,-zc,vz_end,1+step,Tsup+step,dt)
        end
        step=step+Tsup
    
        for s in 1:Tdl
            sx_from_com[:,s+step]=sx_from_w[:,i]-com_trjx[s+step]
            sy_from_com[:,s+step]=sy_from_w[:,i]-com_trjy[s+step]
        end
        step=step+Tdl
    end
end
;
#-1が使われている可能性あり

In [11]:
function generate_csv(dataframe)
    CSV.write("leg_trj.csv",dataframe,delim=',',header=false)
end

generate_csv (generic function with 1 method)

In [12]:
df = DataFrame(xr=sx_from_com[1,:],
                xc=sx_from_com[2,:],
                xl=sx_from_com[3,:],
                yr=sy_from_com[1,:],
                yc=sy_from_com[2,:],
                yl=sy_from_com[3,:],
                zr=sz_from_com[1,:],
                zc=sz_from_com[2,:],
                zl=sz_from_com[3,:])
df=generate_csv(df)

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: Base.Dates.DateFormat(Base.Dates.Slot[Base.Dates.DelimitedSlot{Base.Dates.Year}(Base.Dates.Year,'y',4,"-"),Base.Dates.DelimitedSlot{Base.Dates.Month}(Base.Dates.Month,'m',2,"-"),Base.Dates.DelimitedSlot{Base.Dates.Day}(Base.Dates.Day,'d',2,r"(?=\s|$)")],"","english"),IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1),"leg_trj.csv",0,false,String["xr","xc","xl","yr","yc","yl","zr","zc","zl"],false)